# TP 3 - object detection part using the previously trained model
-> can be run on a PC or on Google Colab
-> you can use a pre-recorded video sequence or the webcam (if run on the PC)

-> load the necessary packages
-> connect google drive to access the model
-> load the model you trained

The code below is ready to use but you need to change the paths

In [1]:
!pip install ultralytics

#import yolov8 libraries
#https://docs.ultralytics.com/

import ultralytics
from ultralytics import YOLO

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# mount google colab
from google.colab import drive
drive.mount('/gdrive')

#on charge notre modèle, entrainé sur notre dataset
#tutorial : https://docs.ultralytics.com/modes/predict/#inference-sources

model = YOLO('/gdrive/MyDrive/IAE/tp3/best.onnx',task='detect') # your trained model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.7/882.7 kB 14.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /gdrive


And now we are going to play a video to test our application and make a prototype of the application

**CPU**

In [13]:
import cv2
from google.colab.patches import cv2_imshow  # Pour Colab

# Taille de l'image (à ajuster selon votre vidéo)
image_width = 224
image_height = 224

# Ouvrir la vidéo
cap = cv2.VideoCapture('/gdrive/MyDrive/IAE/tp3/VID_20231005_225213.mp4')

# Lire la vidéo jusqu'à ce que "q" soit pressé
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        # Redimensionner l'image
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)

        # Prédiction sur l'image
        results = model.predict(frame, device='cpu', imgsz=224)

        # Vérifier s'il y a des résultats de détection
        if len(results[0]) > 0:
            # Dessiner et afficher chaque objet détecté
            for r in results:
                res_plotted = results[0].plot()  # Récupérer les résultats dessinés
                box = r.boxes.cpu()  # Récupérer les bounding boxes (boîtes englobantes)
                box_sz = box.xyxy.numpy()  # Extraire les coordonnées (xmin, ymin, xmax, ymax)

                # Si un objet est détecté (au moins une boîte)
                if box_sz.size != 0:
                    # Récupérer la première boîte détectée
                    x_min, y_min, x_max, y_max = box_sz[0]

                    # Calcul du centre de la balle
                    x_center_ball = int((x_min + x_max) / 2)
                    y_center_ball = int((y_min + y_max) / 2)

                    # Calcul du centre de l'image
                    x_center_image = image_width // 2
                    y_center_image = image_height // 2

                    # Calcul des distances de déplacement
                    delta_x = x_center_ball - x_center_image
                    delta_y = y_center_ball - y_center_image

                    # Dessiner les cercles
                    r = int((y_max - y_min) / 2)
                    cv2.circle(res_plotted, (x_center_ball, y_center_ball), r, (0, 255, 0), 2)  # Cercle autour de la balle
                    cv2.circle(res_plotted, (x_center_ball, y_center_ball), 4, (0, 0, 255), -1)  # Centre de la balle
                    cv2.circle(res_plotted, (x_center_image, y_center_image), 4, (255, 0, 0), -1)  # Centre de l'image

                    # Dessiner une ligne entre le centre de l'image et le centre de la balle
                    cv2.line(res_plotted, (x_center_image, y_center_image), (x_center_ball, y_center_ball), (255, 255, 255), 2)

                    # Ajouter le texte des déplacements sur l'image
                    text_x = f"delta_x: {delta_x}"
                    text_y = f"delta_y: {delta_y}"
                    cv2.putText(res_plotted, text_x, (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                    cv2.putText(res_plotted, text_y, (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Afficher l'image avec les résultats tracés
            cv2_imshow(res_plotted)

        # Quitter si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

**GPU**

In [14]:
import cv2
from google.colab.patches import cv2_imshow  # Pour Colab

# Taille de l'image (à ajuster selon votre vidéo)
image_width = 224
image_height = 224

# Ouvrir la vidéo
cap = cv2.VideoCapture('/gdrive/MyDrive/IAE/tp3/VID_20231005_225213.mp4')

# Lire la vidéo jusqu'à ce que "q" soit pressé
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        # Redimensionner l'image
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)

        # Prédiction sur l'image
        results = model.predict(frame, device='gpu', imgsz=224)

        # Vérifier s'il y a des résultats de détection
        if len(results[0]) > 0:
            # Dessiner et afficher chaque objet détecté
            for r in results:
                res_plotted = results[0].plot()  # Récupérer les résultats dessinés
                box = r.boxes.cpu()  # Récupérer les bounding boxes (boîtes englobantes)
                box_sz = box.xyxy.numpy()  # Extraire les coordonnées (xmin, ymin, xmax, ymax)

                # Si un objet est détecté (au moins une boîte)
                if box_sz.size != 0:
                    # Récupérer la première boîte détectée
                    x_min, y_min, x_max, y_max = box_sz[0]

                    # Calcul du centre de la balle
                    x_center_ball = int((x_min + x_max) / 2)
                    y_center_ball = int((y_min + y_max) / 2)

                    # Calcul du centre de l'image
                    x_center_image = image_width // 2
                    y_center_image = image_height // 2

                    # Calcul des distances de déplacement
                    delta_x = x_center_ball - x_center_image
                    delta_y = y_center_ball - y_center_image

                    # Dessiner les cercles
                    r = int((y_max - y_min) / 2)
                    cv2.circle(res_plotted, (x_center_ball, y_center_ball), r, (0, 255, 0), 2)  # Cercle autour de la balle
                    cv2.circle(res_plotted, (x_center_ball, y_center_ball), 4, (0, 0, 255), -1)  # Centre de la balle
                    cv2.circle(res_plotted, (x_center_image, y_center_image), 4, (255, 0, 0), -1)  # Centre de l'image

                    # Dessiner une ligne entre le centre de l'image et le centre de la balle
                    cv2.line(res_plotted, (x_center_image, y_center_image), (x_center_ball, y_center_ball), (255, 255, 255), 2)

                    # Ajouter le texte des déplacements sur l'image
                    text_x = f"delta_x: {delta_x}"
                    text_y = f"delta_y: {delta_y}"
                    cv2.putText(res_plotted, text_x, (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                    cv2.putText(res_plotted, text_y, (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Afficher l'image avec les résultats tracés
            cv2_imshow(res_plotted)

        # Quitter si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.